## super 没那么简单
http://python.jobbole.com/87291/

Python 3 和 Python 2 的另一个区别是: Python 3 可以使用直接使用 super().xxx 代替 super(Class, self).xxx 

In [2]:
# 默认， Python 3
class A:
    pass

# 默认，Python 3
class B(A):
    def add(self, x):
        super().add(x)

## 单继承

在单继承中 super 就像大家所想的那样，主要是用来调用父类的方法的。

In [19]:
class A:
    def __init__(self):
        self.n = 2
 
    def add(self, m):
        print('self is {0} @A.add and self.n = {1}'.format(self, self.n))
        self.n += m
 
 
class B(A):
    def __init__(self):
        self.n = 3
 
    def add(self, m):
        print('self is {0} @B.add and self.n = {1}'.format(self, self.n))
        super().add(m)
        print('between super().add and self.add: self.n = {}'.format(self.n))
        self.n += 3

In [17]:
b = B()
print(b.n)
b.add(2)
print(b.n)

3
self is <__main__.B object at 0x1045a2320> @B.add and self.n = 3
self is <__main__.B object at 0x1045a2320> @A.add and self.n = 3
between super().add and self.add: self.n = 5
8


## 多继承

这次我们再定义一个 class C，一个 class D:

In [20]:
class C(A):
    def __init__(self):
        self.n = 4
 
    def add(self, m):
        print('self is {0} @C.add'.format(self))
        super().add(m)
        self.n += 4
 
 
class D(B, C):
    def __init__(self):
        self.n = 5
 
    def add(self, m):
        print('self is {0} @D.add'.format(self))
        super().add(m)
        self.n += 5

In [21]:
d = D()
d.add(2)
print(d.n)

self is <__main__.D object at 0x1048308d0> @D.add
self is <__main__.D object at 0x1048308d0> @B.add and self.n = 5
self is <__main__.D object at 0x1048308d0> @C.add
self is <__main__.D object at 0x1048308d0> @A.add and self.n = 5
between super().add and self.add: self.n = 11
19


## super 是个类

当我们调用 super() 的时候，实际上是实例化了一个 super 类。你没看错， super 是个类，既不是关键字也不是函数等其他数据结构:

In [1]:
class A: pass
s = super(A)
type(s)

super

在大多数情况下， super 包含了两个非常重要的信息: 一个 MRO 以及 MRO 中的一个类。

当以如下方式调用 super 时, MRO 指的是 type(obj) 的 MRO, MRO 中的那个类就是 a_type , 同时 isinstance(obj, a_type) == True 。

当这样调用时, MRO 指的是 type2 的 MRO, MRO 中的那个类就是 type1 ，同时 issubclass(type2, type1) == True 。

那么， super() 实际上做了啥呢？简单来说就是：提供一个 MRO 以及一个 MRO 中的类 C ， super() 将返回一个从 MRO 中 C 之后的类中查找方法的对象。

也就是说，查找方式时不是像常规方法一样从所有的 MRO 类中查找，而是从 MRO 的 tail 中查找。

## 多继承中 super 的工作方式

下面我们来具体分析一下前面的例子:

D 的 MRO 是: [D, B, C, A, object] 。 

Note: 可以通过 D.mro() (Python 2 使用 D.__mro__ ) 来查看 D 的 MRO 信息）

In [13]:
class A:
    def __init__(self):
        self.n = 2
 
    def add(self, m):
        # 第四步
        # 来自 D.add 中的 super
        # self == d, self.n == d.n == 5
        print('self is {0} @A.add'.format(self))
        self.n += m
        # d.n == 7
 
 
class B(A):
    def __init__(self):
        self.n = 3
 
    def add(self, m):
        # 第二步
        # 来自 D.add 中的 super
        # self == d, self.n == d.n == 5
        print('self is {0} @B.add'.format(self))
        # 等价于 suepr(B, self).add(m)
        # self 的 MRO 是 [D, B, C, A, object]
        # 从 B 之后的 [C, A, object] 中查找 add 方法
        super().add(m)
 
        # 第六步
        # d.n = 11
        self.n += 3
        # d.n = 14
 
class C(A):
    def __init__(self):
        self.n = 4
 
    def add(self, m):
        # 第三步
        # 来自 B.add 中的 super
        # self == d, self.n == d.n == 5
        print('self is {0} @C.add'.format(self))
        # 等价于 suepr(C, self).add(m)
        # self 的 MRO 是 [D, B, C, A, object]
        # 从 C 之后的 [A, object] 中查找 add 方法
        super().add(m)
 
        # 第五步
        # d.n = 7
        self.n += 4
        # d.n = 11
 
 
class D(B, C):
    def __init__(self):
        self.n = 5
 
    def add(self, m):
        # 第一步
        print('self is {0} @D.add'.format(self))
        # 等价于 super(D, self).add(m)
        # self 的 MRO 是 [D, B, C, A, object]
        # 从 D 之后的 [B, C, A, object] 中查找 add 方法
        super().add(m)
        # 第七步
        # d.n = 14
        self.n += 5
        # self.n = 19

        
print('D.mro(): \n{}'.format(D.mro()))
print()
d = D()
d.add(2)
print(d.n)

D.mro(): 
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]

self is <__main__.D object at 0x10469f438> @D.add
self is <__main__.D object at 0x10469f438> @B.add
self is <__main__.D object at 0x10469f438> @C.add
self is <__main__.D object at 0x10469f438> @A.add
19


调用过程图如下: